## Import the using package

In [25]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import requests
import numpy as np
import sklearn
import monpa
import random
import pickle
import json

In [26]:
class Bi_GRU_Model (nn.Module):
    def __init__(self, total_tag, embeding_size, gru_hidden,gru_layer, output_size):
        super(Bi_GRU_Model, self).__init__()
        self.total_tag = total_tag
        self.embeding_size = embeding_size
        self.gru_hidden = gru_hidden
        self.gru_layer = gru_layer
        self.output_size = output_size
        self.Embeding = nn.Embedding(self.total_tag, self.embeding_size)
        self.GRU = nn.GRU(self.embeding_size , self.gru_hidden, self.gru_layer, bidirectional= True)
        
        # batch size is  set to 1 , output_size is set to 2
        self.linear = nn.Linear( self.gru_hidden*2 , self.output_size )
        
    def forward(self, input_data):  
        embed_res = self.Embeding (input_data)
        #print ('Embedding :::: ' , embed_res.size())
        # let the input dimension be the (L , N , M) and the get the output with the dimension (L , N , H)
        gru_res, _ = self.GRU(embed_res.unsqueeze(1))
        #print ("GRU :::: ", gru_res.size())
        linear_res = self.linear(gru_res[-1])
        final_res = F.log_softmax(linear_res)
        #print ('final res :::: ' , final_res)
        return final_res



In [27]:
model_v1 = Bi_GRU_Model(total_tag=16, embeding_size=8, gru_hidden= 6 , gru_layer=1, output_size=7 )
test_vec = torch.LongTensor([1,2,3,4,5,6,7])
model_v1(test_vec)
print ('V1_Model_with_only_gru ::: \n', model_v1)

V1_Model_with_only_gru ::: 
 Bi_GRU_Model(
  (Embeding): Embedding(16, 8)
  (GRU): GRU(8, 6, bidirectional=True)
  (linear): Linear(in_features=12, out_features=7, bias=True)
)


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Data Pre-processing

* 將positive_data和negative_data載入存入List
* 將monpa的tags載入dictionary
* 將sentence 的 words 轉成詞性


In [28]:
'''training_data_sentence_list = list()
training_data_tag_list = list()
training_target_sentence_list = list()
dict_idx2tag = dict()
dict_tag2idx = dict()

# Read both the postive data and negative data
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))
        training_target_sentence_list.append(1)
with open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:
    for pos_s in rf.readlines():
        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))
        training_target_sentence_list.append(0)
        
print ('training data size : ',len(training_data_sentence_list))
print ('training data target size : ',len(training_target_sentence_list))
tmp_cnt = 0
# Change the words to tags, because we need to use the sequential tags for training.
for sentence in training_data_sentence_list:
    tmp_cnt += 1
    #print (tmp_cnt)
    tmp = monpa.pseg(sentence)
    tmp_list = list()
    for item in tmp:
        tmp_list.append(item[1])
    training_data_tag_list.append(tmp_list)
    
print (len(training_data_tag_list))

# Read the tags of the monpa
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  

print ('The dictionary of idxs 2 tags : ', dict_idx2tag)
print ('The dictionary of tags 2 idxs : ' , dict_tag2idx)

shuffle_zip = list(zip(training_data_tag_list, training_target_sentence_list))
random.shuffle(shuffle_zip)
training_data_tag_list[:], training_target_sentence_list[:] = zip(*shuffle_zip)
print (training_data_tag_list, training_target_sentence_list)
'''


"training_data_sentence_list = list()\ntraining_data_tag_list = list()\ntraining_target_sentence_list = list()\ndict_idx2tag = dict()\ndict_tag2idx = dict()\n\n# Read both the postive data and negative data\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('、').strip('-'))\n        training_target_sentence_list.append(1)\nwith open ('../CJX_Train_Test_Data/positive_data.txt', 'r', encoding = 'utf-8') as rf:\n    for pos_s in rf.readlines():\n        training_data_sentence_list.append(pos_s.strip('\n').strip(',').strip('，').strip('《').strip('》').strip('【').strip('】').strip('、').strip('-'))\n        training_target_sentence_list.append(0)\n        \nprint ('training data size : ',len(training_data_sentence_list))\nprint ('training data target size : ',len(training_target_sen

## Save the training data list

In [29]:
'''f = open ('../pickle/training_tags.pkl', 'wb')
pickle.dump(training_data_tag_list, f)
f.close()
f = open ('../pickle/target.pkl', 'wb')
pickle.dump(training_target_sentence_list, f)
f.close()'''

"f = open ('../pickle/training_tags.pkl', 'wb')\npickle.dump(training_data_tag_list, f)\nf.close()\nf = open ('../pickle/target.pkl', 'wb')\npickle.dump(training_target_sentence_list, f)\nf.close()"

In [30]:
f = open ('../pickle/training_tags.pkl', 'rb')
a = pickle.load( f)
f.close()
f = open ('../pickle/target.pkl', 'rb')
b = pickle.load(f)
f.close()

In [31]:
dict_idx2tag = dict()
dict_tag2idx = dict()
with open ('./monpa_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    cnt = 0
    for tag in rf.readlines():
        dict_idx2tag[cnt] = tag.strip('\n')
        dict_tag2idx[tag.strip('\n')] = cnt
        cnt += 1  
training_data_tag_list = a
training_target_sentence_list = b

'''
dict_idx2tag = dict()
dict_tag2idx = dict()
with open ('./jieba_tag.txt' , 'r' , encoding = 'utf-8') as rf:
    for i,t in enumerate(rf.readlines()):
        dict_idx2tag[i] = t.strip('\n')
        dict_tag2idx[t.strip('\n')] = i
print (dict_idx2tag)
print (dict_tag2idx)

with open ('../CJX_Train_Test_Data/positive_data.txt' , 'r' , encoding = 'utf-8') as rf : 
    for s in rf.readlines():
        html = requests.get('http://192.168.10.216/jieba/cut/api/v1.0/' + s.strip('\n')).text
        tmp_d = json.loads(html)
        res = tmp_d['pos']
        print (res)
        for tag in res:
            if tag not in dict_tag2idx.keys():
                dict_tag2idx[tag] = len(dict_tag2idx.keys())
                dict_idx2tag[len(dict_tag2idx.keys())] = tag
with open ('../CJX_Train_Test_Data/negative_data.txt' , 'r' , encoding = 'utf-8') as rf : 
    for s in rf.readlines():
        html = requests.get('http://192.168.10.216/jieba/cut/api/v1.0/' + s.strip('\n')).text
        tmp_d = json.loads(html)
        res = tmp_d['pos']
        print (res)
        for tag in res:
            if tag not in dict_tag2idx.keys():
                dict_tag2idx[tag] = len(dict_tag2idx.keys())
                dict_idx2tag[len(dict_tag2idx.keys())] = tag

print (dict_idx2tag)
print (dict_tag2idx)
'''      


"\ndict_idx2tag = dict()\ndict_tag2idx = dict()\nwith open ('./jieba_tag.txt' , 'r' , encoding = 'utf-8') as rf:\n    for i,t in enumerate(rf.readlines()):\n        dict_idx2tag[i] = t.strip('\n')\n        dict_tag2idx[t.strip('\n')] = i\nprint (dict_idx2tag)\nprint (dict_tag2idx)\n\nwith open ('../CJX_Train_Test_Data/positive_data.txt' , 'r' , encoding = 'utf-8') as rf : \n    for s in rf.readlines():\n        html = requests.get('http://192.168.10.216/jieba/cut/api/v1.0/' + s.strip('\n')).text\n        tmp_d = json.loads(html)\n        res = tmp_d['pos']\n        print (res)\n        for tag in res:\n            if tag not in dict_tag2idx.keys():\n                dict_tag2idx[tag] = len(dict_tag2idx.keys())\n                dict_idx2tag[len(dict_tag2idx.keys())] = tag\nwith open ('../CJX_Train_Test_Data/negative_data.txt' , 'r' , encoding = 'utf-8') as rf : \n    for s in rf.readlines():\n        html = requests.get('http://192.168.10.216/jieba/cut/api/v1.0/' + s.strip('\n')).text\n 

## Hyperparamters_v1

In [41]:
epoch_v1 = 500
learning_rate_v1 = 0.0001
input_size_v1 = len(dict_idx2tag.keys())
gru_hidden_size_v1 = 8
embedding_size_v1 = 16
output_size_v1 = 2
gru_layer_v1 = 5


## 定義訓練function

In [42]:
def train_v1():
    model_v1 = Bi_GRU_Model(total_tag=input_size_v1, 
                            embeding_size = embedding_size_v1,
                            gru_hidden=gru_hidden_size_v1,
                            output_size=output_size_v1,
                            gru_layer = gru_layer_v1)
    optimizer_v1 = torch.optim.SGD(model_v1.parameters(), lr=learning_rate_v1)
    #model_v1 = torch.load('../pickle/model_v1.pt')
    loss_function_v1 = nn.NLLLoss()
    
    for epoch in range(epoch_v1):
        sub_epoch = 0
        total_loss = 0
        for seq_tag_idx in range(len(training_data_tag_list)):
            sub_epoch += 1
            input_data = torch.LongTensor([dict_tag2idx[t] for t in training_data_tag_list[seq_tag_idx]])
            target_data = torch.LongTensor([training_target_sentence_list[seq_tag_idx]])
            out = model_v1(input_data)
            loss = loss_function_v1(out, target_data)
            total_loss += loss.item()
            optimizer_v1.zero_grad()
            loss.backward()
            optimizer_v1.step()
            '''
            if sub_epoch % 100 == 0:
                print ('Epoch '+  str(epoch) + ' '+ str(sub_epoch)+'/' + str(len(training_data_tag_list)) + '  Loss : ' + str(loss))
            '''
        print ('Epoch ' + str(epoch) + ' Loss : ' + str (total_loss))
        torch.save(model_v1, '../pickle/model_v1.pt')
    
            
            
            
            
            
            

In [43]:
train_v1()

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Loss : 4901.492028713226


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Bi_GRU_Model. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 1 Loss : 4901.335302174091
Epoch 2 Loss : 4901.323291540146
Epoch 3 Loss : 4901.326871812344
Epoch 4 Loss : 4901.328579843044
Epoch 5 Loss : 4901.328007698059
Epoch 6 Loss : 4901.326201558113
Epoch 7 Loss : 4901.323779463768
Epoch 8 Loss : 4901.321159005165
Epoch 9 Loss : 4901.318470954895
Epoch 10 Loss : 4901.315745532513
Epoch 11 Loss : 4901.313029944897
Epoch 12 Loss : 4901.310348570347
Epoch 13 Loss : 4901.307719647884
Epoch 14 Loss : 4901.305081963539
Epoch 15 Loss : 4901.302484750748
Epoch 16 Loss : 4901.299927353859
Epoch 17 Loss : 4901.297402381897
Epoch 18 Loss : 4901.2949305176735
Epoch 19 Loss : 4901.292462646961
Epoch 20 Loss : 4901.2900159955025
Epoch 21 Loss : 4901.287607848644
Epoch 22 Loss : 4901.285248219967
Epoch 23 Loss : 4901.282868862152
Epoch 24 Loss : 4901.280534923077
Epoch 25 Loss : 4901.278239667416
Epoch 26 Loss : 4901.275938749313
Epoch 27 Loss : 4901.273699522018
Epoch 28 Loss : 4901.271473109722
Epoch 29 Loss : 4901.269277334213
Epoch 30 Loss : 4901.

KeyboardInterrupt: 

In [14]:
def predict():
    model_v1 = Bi_GRU_Model(total_tag=input_size_v1, 
                            embeding_size = embedding_size_v1,
                            gru_hidden=gru_hidden_size_v1,
                            output_size=output_size_v1,
                            gru_layer = gru_layer_v1)
    model_v1 = torch.load('../pickle/model_v1.pt')
    correct  = 0
    

    for seq_tag_idx in range(len(training_data_tag_list)):         
        input_data = torch.LongTensor([dict_tag2idx[t] for t in training_data_tag_list[seq_tag_idx]])
        target_data = torch.LongTensor([training_target_sentence_list[seq_tag_idx]])
        out = model_v1(input_data)
        out = out.squeeze(0)
        if out[0].item() > out[1].item() and target_data[0].item() == 0:
            correct += 1
        elif out[0].item() < out[1].item() and target_data[0].item() == 1:
            correct += 1
        else:
            continue
    return correct/ len(training_data_tag_list)
        



In [16]:
predict()

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.5